In [4]:
import numpy as np
import scipy as sp
from IPython.display import clear_output

def get_bound(index1, index2, bonds):
    bond = frozenset({tuple(index1),tuple(index2)})
    if bond in bonds:
        return bonds[bond]
    else:
        bonds[bond] = np.abs(np.random.standard_normal())
        return bonds[bond]

def increase_entry_by_one(A, j):
    A_modified = A.copy()
    A_modified[j] += 1
    return A_modified
def decrease_entry_by_one(A, j):
    A_modified = A.copy()
    A_modified[j] -= 1
    return A_modified

def get_neighbor(indices,L):
    neighbor_index = []
    for j in range(len(L)):
        if (indices[j] == 0):
            neighbor_index.append(increase_entry_by_one(indices, j))
            indice_copy = indices.copy()
            indice_copy[j] = L[j]-1
            neighbor_index.append(indice_copy)
        elif (indices[j] == L[j]-1):
            neighbor_index.append(decrease_entry_by_one(indices, j))
            indice_copy = indices.copy()
            indice_copy[j] = 0
            neighbor_index.append(indice_copy)
        else:
            neighbor_index.append(increase_entry_by_one(indices, j))
            neighbor_index.append(decrease_entry_by_one(indices, j))
    return neighbor_index

def get_energy(spin, spin_index, neighbor_index, S):
    energy = 0
    for neighbor in neighbor_index:
        bond = get_bound(spin_index, neighbor, bonds)
        energy = energy + bond*spin*S[tuple(neighbor)]
    return energy

def overlap(S1,S2,N):
    return np.sum(S1*S2)/N

def sweep(S,L,N):
    sweep = 0
    while sweep < N:
        indices = [np.random.choice(dim) for dim in L]
        spin = S[tuple(indices)]
        neighbor_index = get_neighbor(indices,L)
        beforeE = get_energy(spin, indices, neighbor_index, S)
        afterE = get_energy(-spin, indices, neighbor_index, S)
        deltaE = afterE - beforeE
        if deltaE > 0:
            S[tuple(indices)] = -spin
        sweep = sweep+1

def is_active(index, S, L):
    spin = S[tuple(index)]
    neighbor_index = get_neighbor(np.asarray(index),L)
    beforeE = get_energy(spin, index, neighbor_index, S)
    afterE = get_energy(-spin, index, neighbor_index, S)
    deltaE = afterE - beforeE
    return deltaE > 0

def get_active(S,L):
    it = np.nditer(S,flags = ['multi_index'])
    active_indices = []
    while not it.finished:
        index = it.multi_index
        if is_active(index, S, L):
            active_indices.append(index)
        it.iternext()
    return active_indices

import random

def kineticMonteCarlo(S,L,active_list):
    l = len(active_list)
    if l == 0:
        return 0
    t = 1/l
    index = random.choice(active_list)
    spin = S[tuple(index)]
    neighbor_index = get_neighbor(np.asarray(index),L)
    beforeE = get_energy(spin, index, neighbor_index, S)
    afterE = get_energy(-spin, index, neighbor_index, S)
    deltaE = afterE - beforeE
    if deltaE > 0:
        S[tuple(index)] = -spin
        active_list.remove(tuple(index))
        for nspin in neighbor_index:
            if is_active(nspin,S,L):
                if not (tuple(nspin) in active_list):
                    active_list.append(tuple(nspin))
            else:
                if (tuple(nspin) in active_list):
                    active_list.remove(tuple(nspin))
    return t


In [18]:
def montecarlmethod(S,L,N):
    survival1 = 0
    survival2 = 0
    S2 = S.copy()
    S1 = S.copy()
    bonds = dict()
    while True:
        sweep(S1,L,N)
        survival1 = survival1 + 1
        print('S1 Sweep {} done'.format(survival1))
        clear_output(wait=True)
        if survival1 == 10:
            break

    S1_active = get_active(S1,L)

    while True:
        k = kineticMonteCarlo(S1,L,S1_active)
        if k == 0:
            break
        survival1 = survival1 + k
    print('S1 done')
    clear_output(wait=True)

    while True:
        sweep(S2,L,N)
        survival2 = survival2 + 1
        print('S2 Sweep {} done'.format(survival2))
        clear_output(wait=True)
        if survival2 == 10:
            break

    S2_active = get_active(S2,L)

    while True:
        k = kineticMonteCarlo(S2,L,S2_active)
        if k == 0:
            break
        survival2 = survival2 + k

    return [overlap(S1,S2,N),survival1,survival2] 

In [10]:
L = np.array([100, 100])
N = np.prod(L)
S = np.random.choice([-1, 1], size=tuple(L))
bonds = dict()
print(montecarlmethod(S,L,N))

[0.4316, 12.0, 13.0]


In [11]:
L = np.array([100, 35, 35])
N = np.prod(L)
S = np.random.choice([-1, 1], size=tuple(L))
bonds = dict()
print(montecarlmethod(S,L,N))

[0.3718530612244898, 29.029130128121043, 25.4986379374633]


In [12]:
L = np.array([100, 10, 10, 10])
N = np.prod(L)
S = np.random.choice([-1, 1], size=tuple(L))
bonds = dict()
print(montecarlmethod(S,L,N))

[0.34626, 39.49708852260452, 57.85361678147303]


In [14]:
L = np.array([100, 6, 6, 6, 6])
N = np.prod(L)
S = np.random.choice([-1, 1], size=tuple(L))
bonds = dict()
print(montecarlmethod(S,L,N))

[0.30430555555555555, 101.55073393571195, 142.92602198137592]


In [21]:
L = np.array([100, 5, 5, 4, 4, 4])
N = np.prod(L)
S = np.random.choice([-1, 1], size=tuple(L))
bonds = dict()
print(montecarlmethod(S,L,N))

[0.569375, 89.31799334982644, 96.54443373708355]


In [22]:
L = np.array([100, 4, 4, 4, 3, 3, 3])
N = np.prod(L)
S = np.random.choice([-1, 1], size=tuple(L))
bonds = dict()
print(montecarlmethod(S,L,N))

[0.465, 94.31193977296284, 71.50809814126035]


In [23]:
L = np.array([100, 3, 3, 3, 3, 3, 3, 3])
N = np.prod(L)
S = np.random.choice([-1, 1], size=tuple(L))
bonds = dict()
print(montecarlmethod(S,L,N))

[0.2566255144032922, 65.39378476379832, 96.61115001001197]


In [24]:
L = np.array([100, 3, 3, 3, 3, 3, 3, 2, 2])
N = np.prod(L)
S = np.random.choice([-1, 1], size=tuple(L))
bonds = dict()
print(montecarlmethod(S,L,N))

[0.5414471879286694, 93.4477045659835, 126.86242806076645]


In [26]:
L = np.array([100, 3, 3, 3, 3, 3, 2, 2, 2, 2])
N = np.prod(L)
S = np.random.choice([-1, 1], size=tuple(L))
bonds = dict()
print(montecarlmethod(S,L,N))

[0.31286522633744857, 85.5088152534387, 89.95189111186777]
